In [5]:
import matplotlib.pyplot as plt
from task1 import *
from task2 import *
from task3 import *
from task4 import *
import cfpq_data



[2024-03-20 18:39:25]>INFO>Found graph with name='bzip'
[2024-03-20 18:39:27]>INFO>Load archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/bzip.tar.gz')
[2024-03-20 18:39:27]>INFO>Unzip graph name='bzip' to file graph=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/bzip/bzip.csv')
[2024-03-20 18:39:27]>INFO>Remove archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/bzip.tar.gz')
[2024-03-20 18:39:27]>INFO>Load graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x1273b9040> from path=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/bzip/bzip.csv')
[2024-03-20 18:39:27]>INFO>Found graph with name='bzip'
[2024-03-20 18:39:30]>INFO>Load archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/bzip.tar.gz')
[2024-03-20 18:39:30]>INFO>Unzip graph name='bzip' to file graph=PosixPath('/usr/local/lib/python3.9/site-pack

Number of nodes: 632
Number of edges: 556
Labels: {'a', 'd'}


In [17]:
graphs = {}
graph_names = ["bzip", "ls", "gzip"]

for graph_name in graph_names:
    graph_path = cfpq_data.download(graph_name)
    graph = cfpq_data.graph_from_csv(graph_path)
    nNodes, nEdges, labels = graph_info(graph_name)
    graphs[graph_name] = {}
    graphs[graph_name]["graph"] = graph
    graphs[graph_name]["nNodes"] = nNodes
    graphs[graph_name]["nEdges"] = nEdges
    graphs[graph_name]["labels"] = labels
    print(graph_name, graph_info(graph_name))
    
graph_queries = {}
graph_queries["bzip"] = ["(a | d)* a", "(a | d)+ d*", "a d a (a|d)*", "(a | d) (a | d)"]
graph_queries["ls"] = ["(a | d)* a", "(a | d)+ d*", "a d a (a|d)*", "(a | d) (a | d)"]
graph_queries["gzip"] = ["(a | d)* a", "(a | d)+ d*", "a d a (a|d)*", "(a | d) (a | d)"]

[2024-03-20 19:25:51]>INFO>Found graph with name='bzip'
[2024-03-20 19:25:53]>INFO>Load archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/bzip.tar.gz')
[2024-03-20 19:25:53]>INFO>Unzip graph name='bzip' to file graph=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/bzip/bzip.csv')
[2024-03-20 19:25:53]>INFO>Remove archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/bzip.tar.gz')
[2024-03-20 19:25:53]>INFO>Load graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x1273ed130> from path=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/bzip/bzip.csv')
[2024-03-20 19:25:53]>INFO>Found graph with name='bzip'
[2024-03-20 19:25:55]>INFO>Load archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/bzip.tar.gz')
[2024-03-20 19:25:55]>INFO>Unzip graph name='bzip' to file graph=PosixPath('/usr/local/lib/python3.9/site-pack

bzip (632, 556, {'a', 'd'})


[2024-03-20 19:25:58]>INFO>Load archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/ls.tar.gz')
[2024-03-20 19:25:58]>INFO>Unzip graph name='ls' to file graph=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/ls/ls.csv')
[2024-03-20 19:25:58]>INFO>Remove archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/ls.tar.gz')
[2024-03-20 19:25:58]>INFO>Load graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x1273b26d0> from path=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/ls/ls.csv')
[2024-03-20 19:25:58]>INFO>Found graph with name='ls'
[2024-03-20 19:26:00]>INFO>Load archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/ls.tar.gz')
[2024-03-20 19:26:00]>INFO>Unzip graph name='ls' to file graph=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/ls/ls.csv')
[2024-03-20 19:26:00]>INFO>Remove arc

ls (1687, 1453, {'a', 'd'})


[2024-03-20 19:26:04]>INFO>Load archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/gzip.tar.gz')
[2024-03-20 19:26:04]>INFO>Unzip graph name='gzip' to file graph=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/gzip/gzip.csv')
[2024-03-20 19:26:04]>INFO>Remove archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/gzip.tar.gz')
[2024-03-20 19:26:04]>INFO>Load graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x1273c26a0> from path=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/gzip/gzip.csv')
[2024-03-20 19:26:04]>INFO>Found graph with name='gzip'
[2024-03-20 19:26:07]>INFO>Load archive graph_archive=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/gzip.tar.gz')
[2024-03-20 19:26:07]>INFO>Unzip graph name='gzip' to file graph=PosixPath('/usr/local/lib/python3.9/site-packages/cfpq_data/data/graphs/gzip/gzip.csv')
[2024-03-20 1

gzip (2687, 2293, {'a', 'd'})


In [18]:
# source_vertices = cfpq_data.generate_multiple_source(graphs["bzip"], 10, seed=42)

In [ ]:
for graph_name in graph_names:
    for query in graph_queries[graph_name]:
        print(graph_name, query)
        for number_of_start_nodes in range(graphs[graph_name]["nNodes"]):
            start_nodes = cfpq_data.generate_multiple_source(graphs[graph_name]["graph"], number_of_start_nodes, seed = 42)
            final_nodes = cfpq_data.generate_multiple_source(graphs[graph_name]["graph"], graphs[graph_name]["nNodes"]//2, seed = 42)
            

            
            
    
    